# Deep Hedging AI 4.0 Engine
### Vanilla Deep Hedging engine reference implementation with dynamic training update.

This is the main example notebook. It shows learning to hedge a vanilla call option (ATM by default), first in a Black & Scholes world with statistical drift, and secondly in a world where a second option can be traded.
The examples are not intended to be overly realistic.

In the Black & Scholes case we see that the hedge learned is _not_ the risk-neutral hedge. The notebook <tt>trainer-bs_nodrift.ipynb</tt> demonstrates that if the statistical drift is zero, and step size is sufficiently small (daily), then the Deep Hedging hedge approximates the risk-neutral hedge.

### Hans Buehler, March 26 2023


In [1]:
""" Slighly annoying: by default the SageMaker Python import directory does not include our git directory """
#!pip -q install cdxbasics "tensorflow>=2.11" "tensorflow_probability==0.19"
import os
p = os.getcwd()
dhn = "/deephedging/"
i = p.find(dhn)
if i!=-1:
    p = p[:i]
    import sys
    sys.path.append(p)
    print("SageMaker: added python path %s" % p)
else:
    print("Not on Sagemaker")

Not on Sagemaker


In [3]:
print("Deep Hedging AI says hello ... ")

import importlib as imp
import packages.cdx_tf.cdx_tf.util as _
imp.reload(_)
import packages.cdx_tf.cdx_tf.gym as _
imp.reload(_)
import packages.cdx_tf.cdx_tf.clip as _
imp.reload(_)
import packages.cdx_tf.cdx_tf.optimizer as _
imp.reload(_)
import packages.cdx_tf.cdx_tf.models as _
imp.reload(_)
import packages.cdx_tf.cdx_tf.monetary_utility as _
imp.reload(_)
import deephedging.v4.gym as _
imp.reload(_)
import deephedging.v4.trainer as _
imp.reload(_)

from cdxbasics.config import Config
from deephedging.v4.trainer import train2
from deephedging.v4.gym import VanillaDeepHedgingGym
from deephedging.v4.world import SimpleWorld_Spot_ATM

from IPython.display import display, Markdown

# see print of the config below for numerous options
config = Config()
# world
config.world.samples = 100
config.world.black_scholes = True
# gym
config.gym.objective.utility = "cvar"
config.gym.objective.lmbda = 1.
config.gym.objective.depth = 2
config.gym.objective.width = 10
config.gym.objective.activation = "softplus"
config.gym.agent.depth = 3
config.gym.agent.width = 10
config.gym.agent.activation = "softplus"
config.gym.init_agent.depth = 2
config.gym.init_agent.width = 10
config.gym.init_agent.activation = "softplus"
# trainer
config.trainer.train.optimizer.name = "adam"
config.trainer.train.optimizer.learning_rate = 0.001
config.trainer.train.optimizer.clipvalue = 1.
config.trainer.train.optimizer.global_clipnorm = 1.
config.trainer.train.batch_size = None
config.trainer.train.epochs = 12
config.trainer.caching.mode = "on"
"""
config.trainer.visual.epoch_refresh = 5
config.trainer.visual.confidence_pcnt_lo = 0.25
config.trainer.visual.confidence_pcnt_hi = 0.75
"""
# create world
world  = SimpleWorld_Spot_ATM( config.world )
val_world  = world.clone(samples=world.nSamples//10)

r = train2( world     = world,
            val_world = val_world,
            config    = config )

# create training environment
print("=========================================")
print("Result keys")
print(r.keys())

print("=========================================")
print("Config usage report")
print( config.usage_report() )
config.done()

Deep Hedging AI says hello ... 
00: Initializing training for VanillaDeepHedging
01:   VanillaDeepHedging: model generated. Preparing training for 12 epochs. Model has 909 trainable weights.
02:     'VanillaDeepHedging is using a total of 909 weights: 431 for the main agent, 196 for 5 initial states and delta, and 282 weights for the two cvar@1 monetary utilities
02:      Features available for the agent per time step:   action, cost, delta, ivol, lbnd_a, pnl, price, spot, sqrt_time_left, time_left and ubnd_a
02:      Features used by the agent per time step:         delta, price and time_left
02:      Features available for the initial agent:         cost, ivol, lbnd_a, price, spot, sqrt_time_left, time_left and ubnd_a
02:      Features used by the initial agent:               price
02:      Features available for the monetary utilities:    cost, ivol, lbnd_a, price, spot, sqrt_time_left, time_left and ubnd_a
02:      Features used by the monetary utilities:          price
02:     Num